In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import json
from tqdm import tqdm_notebook as tqdm
import math
import random
import glob
from tqdm import tqdm_notebook as tqdm

In [3]:
def dump_json(js_meta, name):
    json_data = json.dumps(js_meta)
    f = open(name,"w")
    f.write(json_data)
    f.close()

In [4]:
audio_meta_dir = "metadata_audio_altered.csv"
full_meta_dir = "full_metadata.json"

In [5]:
actually_real_audio = ["yfxtuimubx.mp4","ecumyiowzs.mp4", "ddcfzufmyd.mp4", "axyzsqhnts.mp4", "amrryjxojv.mp4"]

In [6]:
df_audio = pd.read_csv(audio_meta_dir)
print(len(df_audio))
df_audio = df_audio[~df_audio["train_mp4"].isin(actually_real_audio)]
print(len(df_audio))

10157
10152


In [7]:
fake_mp4s = list(df_audio["train_mp4"].values)
list.sort(fake_mp4s)

In [8]:
len(fake_mp4s)

10152

In [9]:
dict_meta =json.load(open(full_meta_dir,'rb'))

In [10]:
len(dict_meta)

119146

In [11]:
wavs = list(glob.iglob("../dltraining/wavs/*.wav"))
wavs = [x.replace("../dltraining/wavs/","").replace(".wav",".mp4") for x in wavs]

In [12]:
no_audio_meta_item = []
for key, v in tqdm(dict_meta.items()):
    if key not in wavs:
        print(key, v)
        no_audio_meta_item.append(key)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


taqnnsyxip.mp4 {'label': 'FAKE', 'split': 'train', 'original': 'pjibpowymk.mp4', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
teehidqtii.mp4 {'label': 'REAL', 'split': 'train', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
pjibpowymk.mp4 {'label': 'REAL', 'split': 'train', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
inkxxqwrzi.mp4 {'label': 'FAKE', 'split': 'train', 'original': 'teehidqtii.mp4', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_5', 'folder_name': 'dfdc_train_part_5'}
clmgkufday.mp4 {'label': 'FAKE', 'split': 'train', 'original': 'uyogufqlec.mp4', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_15', 'folder_name': 'dfdc_train_part_15'}
uyogufqlec.mp4 {'label': 'REAL', 'split': 'train', 'data_dir': '/dltraining/datasets/data/dfdc_train_part_15', 'folder_name': 'dfdc_train_part_15'}
inluuhjteb.mp4 {'label': 'REAL

In [13]:
for nami in no_audio_meta_item:
    del dict_meta[nami]

In [14]:
len(dict_meta)

119116

In [15]:
for key, value in tqdm(dict_meta.items()):
    if key in fake_mp4s:
        value['audio_label'] = 'FAKE' 
        dict_meta[key] =value
    else:
        value['audio_label'] = 'REAL' 
        dict_meta[key] =value

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [16]:
dump_json(dict_meta, "full_audio_meta.json")

In [17]:
train_num = math.ceil(len(fake_mp4s)*0.8)

In [18]:
training_fake_mp4s = fake_mp4s[0:train_num]
valid_fake_mp4s = fake_mp4s[train_num:]
len(training_fake_mp4s), len(valid_fake_mp4s)

(8122, 2030)

In [19]:
all_meta_mp4s = list(dict_meta.keys())

In [20]:
real_mp4s = list(set(all_meta_mp4s)-set(training_fake_mp4s)-set(valid_fake_mp4s))

In [21]:
num_valid = len(valid_fake_mp4s)
valid_real_mp4s = random.sample(real_mp4s,num_valid)
train_real_mp4s = list(set(real_mp4s)-set(valid_real_mp4s))

In [22]:
valid_mp4s = valid_real_mp4s + valid_fake_mp4s
train_mp4s = train_real_mp4s + training_fake_mp4s

In [23]:
len(valid_mp4s),len(train_mp4s)

(4060, 115056)

In [24]:
dict_train_meta = {}
for key, value in tqdm(dict_meta.items()):
    if key in train_mp4s:
        dict_train_meta[key] = value

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [25]:
dict_valid_meta = {}
for key, value in tqdm(dict_meta.items()):
    if key in valid_mp4s:
        dict_valid_meta[key] = value

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [26]:
len(dict_train_meta), len(dict_valid_meta)

(115056, 4060)

In [27]:
dump_json(dict_train_meta, "train_audio_meta.json")
dump_json(dict_valid_meta, "valid_audio_meta.json")